# Notebook #4: Cálculo de Rentabilidad

- Con los datos extraídos y el modelo de *machine learning* que se ha entrenado para predecir el precio del alquiler, somos capaces de realizar el cálculo de rentabilidad con ayuda de distintas funciones.

- Importamos las librerías y soportes.

In [1]:
%load_ext autoreload
%autoreload 2

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import geopandas as gpd
import numpy as np
import pickle
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los dfFrames

from tqdm import tqdm

# Ignorar los warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings('ignore')

# Librería para el acceso a variables y funciones
# -----------------------------------------------------------------------
import sys
sys.path.append("../")
from src import soporte_rentabilidad as sr
from src import soporte_yolo as sy
from src import soporte_scoring as ss
from src import soporte_blip as sb
from src import soporte_mongo as sm

- Como primer paso, definimos una variable con las rutas de los transformers: el encoder, el scaler y el modelo Random Forest.

- También vamos a conectarnos a la base de datos de Mongo para obtener el los datos correspondientes a viviendas en venta.

- Utilizaremos la función `predecir_alquiler` para aplicar el modelo a la totalidad de viviendas en venta, que devuelve el dataframe con una nueva columna 'alquiler_predicho'. Sus únicos argumentos son el archivo pickle con las viviendas en venta y la variable con las rutas de los transformers.

In [2]:
paths_transformers = ["../transformers/target_encoder.pkl", "../transformers/scaler.pkl", "../transformers/model.pkl"]
bd = sm.conectar_a_mongo("ProyectoRentabilidad")
gdf = sm.importar_a_geodataframe(bd, "venta")
gdf = sr.predecir_alquiler(gdf, paths_transformers)

- Comprobamos el dataframe.

- Para cada vivienda, tenemos una columna que corresponde a una lista con las urls de las imágenes. Para poder identificar cuál corresponde a cada estancia, utilizaremos el modelo YOLOv11, en su versión x y de clasificación, por cuanto no nos interesa determinar la ubicación ni la forma de los objetos en las imágenes, solamente su existencia.

- Con la función `identificar_urls_habitaciones`, se añaden etiquetas a la cocina y al baño de cada anuncio. Para optimizar el tiempo de procesamiento, cuando las ha encontrado, no continua con el resto de imágenes, sino que salta al siguiente anuncio.

- El resultado es un dataframe con nuevas columnas para la URL del baño y de la cocina. Cuando uno de los dos valores es nulo, elimina esa fila.
- El return también incluye dos dataframe adicionales: uno con todas las detecciones de las imágenes procesadas y otro con las filas donde, no se encontraron imágenes del baño o la cocina, como una forma de poder validar los resultados.

In [10]:
gdf_tags, df_detecciones, df_nulos = sy.identificar_urls_habitaciones(gdf, 'urls_imagenes', drop_nulls=False)

100%|██████████| 340/340 [48:35<00:00,  8.58s/it] 


- Vamos a comprobar el resultado del filtrado.

- Además del dataframe con las imágenes filtradas, tenemos un segundo dataframe como retorno, que almacena los elementos que se han detectado en cada imagen, la URL y si ha sido etiquetado como cocina o baño. Esto nos permite realizar generar una métrica sobre la efectividad de YOLO.

- Si quisiéramos directamente eliminar las propiedades donde no se han encontrado imágenes de la cocina y el baño, podríamos utilizar el parámetro *drop nulls*.

In [11]:
#gdf_tags.to_pickle("../data/transformed/final_tags.pkl")

In [12]:
gdf_tags.shape[0]

340

In [13]:
df_detecciones[df_detecciones['habitación']=='kitchen'].head(5)

,url,detecciones,habitación
2,https://img4.idealista.com/blur/WEB_LISTING-M/...,"[microwave, refrigerator, wardrobe, dishwasher...",kitchen
7,https://img4.idealista.com/blur/WEB_LISTING-M/...,"[microwave, washbasin, refrigerator, washer, s...",kitchen
14,https://img4.idealista.com/blur/WEB_LISTING-M/...,"[microwave, refrigerator, dishwasher, quilt, w...",kitchen
15,https://img4.idealista.com/blur/WEB_LISTING-M/...,"[microwave, dishwasher, refrigerator, Crock_Po...",kitchen
25,https://img4.idealista.com/blur/WEB_LISTING-M/...,"[microwave, dishwasher, stove, medicine_chest,...",kitchen


- Al comprobar las URLs de forma manual utilizando una muestra aleatoria, se confirma una accuracy de entorno al 90% en el etiquetado.

In [14]:
for value in df_detecciones[df_detecciones['habitación']=='kitchen']['url'].sample(10):
    print(value)

https://img4.idealista.com/blur/WEB_LISTING-M/0/id.pro.es.image.master/17/32/4a/1215307176.webp
https://img4.idealista.com/blur/WEB_LISTING-M/0/id.pro.es.image.master/ac/a3/bd/1300494221.webp
https://img4.idealista.com/blur/WEB_LISTING-M/0/id.pro.es.image.master/32/8f/eb/1305661215.webp
https://img4.idealista.com/blur/WEB_LISTING-M/0/id.pro.es.image.master/8d/d8/56/1299109536.webp
https://img4.idealista.com/blur/WEB_LISTING-M/0/id.pro.es.image.master/87/42/42/1296914853.webp
https://img4.idealista.com/blur/WEB_LISTING-M/0/id.pro.es.image.master/66/dd/48/1280571979.webp
https://img4.idealista.com/blur/WEB_LISTING-M/0/id.pro.es.image.master/7b/c1/a7/1217126092.webp
https://img4.idealista.com/blur/WEB_LISTING-M/0/id.pro.es.image.master/3e/97/7f/1186509320.webp
https://img4.idealista.com/blur/WEB_LISTING-M/0/id.pro.es.image.master/82/8f/f1/1298976665.webp
https://img4.idealista.com/blur/WEB_LISTING-M/0/id.pro.es.image.master/10/cf/db/1000291810.webp


- Vamos a asignar la columna URL y visitar los enlaces del `df_nulos`. Revisando los anuncios donde no se encontraron imágenes del baño o de la cocina, vemos que, en muchos de ellos, no se han detectado los elementos por la baja calidad de las imágenes.

In [15]:
df_nulos = sy.asignar_URL(df_nulos)
for value in df_nulos['URL'].sample(10):
    print(value)

https://www.idealista.com/inmueble/97797123/
https://www.idealista.com/inmueble/106414865/
https://www.idealista.com/inmueble/107056611/
https://www.idealista.com/inmueble/107147759/
https://www.idealista.com/inmueble/107250899/
https://www.idealista.com/inmueble/107144617/
https://www.idealista.com/inmueble/104913712/
https://www.idealista.com/inmueble/106808048/
https://www.idealista.com/inmueble/86818261/
https://www.idealista.com/inmueble/106669843/


- Como una capa de verificación adicional, vamos a utilizar el modelo BLIP de Salesforce, que crea descripciones de las imágenes, en este caso, de las URLs identificadas por YOLO. Contamos luego el número de ocurrencias de 'kitchen' o 'bathroom'.

In [20]:
captions_cocinas = sb.generar_descripciones(gdf_tags, 'url_cocina')
sb.contar_palabras(captions_cocinas, 'kitchen')

340it [03:37,  1.57it/s]


235

In [21]:
captions_banios = sb.generar_descripciones(gdf_tags, 'url_banio')
sb.contar_palabras(captions_banios, 'bathroom')

340it [03:46,  1.50it/s]


259

In [27]:
print(f"""Tags cocina: {gdf_tags['url_cocina'].count()}, porcentaje del DF: {round(gdf_tags['url_cocina'].count()/gdf_tags.shape[0]*100)}%
Tags baño: {gdf_tags['url_banio'].count()}, porcentaje del DF: {round(gdf_tags['url_banio'].count()/gdf_tags.shape[0]*100,0)}%
      """)

Tags cocina: 264, porcentaje del DF: 78%
Tags baño: 279, porcentaje del DF: 82.0%
      


- Podemos ver que YOLO, identificó cocinas en el 78% de los anuncios y baños en el 82%.

- De las 264 imágenes identificadas como cocinas por YOLO, BLIP ha coincidido en 234, un 89%. El dato en el caso de los baños mejora, habiendo encontrado 250 coincidencias, un 97%.

- Conocedores de que, habrá falsos negativos y falsos positivos en ambos modelos, el alto porcentaje de coincidencias en ambos casos nos indica que la fiabilidad del primer cribado es alta.

- De las 365 viviendas en venta, se han encontrado imágenes de baño y cocina para 266. Dado que el cliente objetivo del proyecto no es quién se dedica a reformas, sin poder conocer el estado de estas estancias, el cálculo de rentabilidad puede cambiar significativamente por ser los espacios que cuesta más dinero reformar. Por eso, vamos a asignar una puntuación de 0 cuando no haya imagen de la cocina o del baño. También asignaremos un valor de 0m2 si no existe la URL.

- El siguiente paso será utilizar la API de Anthropic, y específicamente el modelo Sonnet, para asignar una puntuación del 1 al 10 al estado de la estancia, y una estimación de los metros cuadrados, que es una consideración que ha surgido durante el desarrollo y la fase de pruebas del proyecto.

- El prompt de sistema es similar a este:

<div style="background-color:rgb(149, 222, 237); padding: 10px; border-left: 6px solid #000080; color: black; border-radius: 10px;">
You are an AI image analysis system specialized in evaluating property conditions. Your task is to analyze multiple property images in batch and provide a precise evaluation.
Instructions for Analysis:

Analyze each pair of images:

Image 1: Kitchen
Image 2: Bathroom

Evaluation Criteria:

Ratings: Whole numbers 1-5
1: Very poor (complete renovation required)
2: Poor (major renovations needed)
3: Fair (some renovations needed)
4: Good (minor improvements required)
5: Excellent (no renovations needed)

Sizes: Whole numbers in square meters (m²)
</div>

- Como puede verse en el prompt, el resultado de la consulta son 4 valores separados por comas, con la función `analizar_propiedades`, separaremos esos resultados y los asignaremos a 4 nuevas columnas del dataframe. Para tener consultas más eficientes, se enviarán en lotes, o *batches*, que corresponden a grupos de viviendas. Este batch más el dataframe con las URLs identificadas son los parámetros de la función.

In [30]:
df_scoring, resultados = ss.analizar_propiedades(gdf_tags, batch=3)

Procesando lotes:   0%|          | 0/99 [00:00<?, ?it/s]

- Vamos a comprobar que el scoring se haya ejecutado correctamente, imprimiendo las primeras filas.

In [31]:
df_scoring.head()

,codigo,precio,precio_por_zona,tipo,exterior,planta,ascensor,tamanio,habitaciones,banios,aire_acondicionado,trastero,terraza,patio,parking,estado,direccion,descripcion,anunciante,contacto,urls_imagenes,distrito,geometry,alquiler_predicho,url_cocina,url_banio,puntuacion_cocina,puntuacion_banio,mts_cocina,mts_banio
0,104792745,149900.0,1180.0,piso,True,4,True,127.0,3,2,False,False,False,True,True,good,"carretera de Huesca, 21","Junto a la Academia General Militar, en carret...","Fincas Ruiz, Jose",876 21 08 84,['https://img4.idealista.com/blur/WEB_LISTING-...,Distrito Rural,POINT (-0.86935 41.6973),1180.0,https://img4.idealista.com/blur/WEB_LISTING-M/...,https://img4.idealista.com/blur/WEB_LISTING-M/...,4,4,15,6
1,105844791,149900.0,1162.0,piso,True,4,True,129.0,3,2,False,False,False,False,True,good,carretera de Huesca,Financiación hipoteca 100%. ¡oportunidad única...,"Ciz Inmobiliaria, CIZ",876 21 02 72,['https://img4.idealista.com/blur/WEB_LISTING-...,Distrito Rural,POINT (-0.86946 41.69574),1186.0,https://img4.idealista.com/blur/WEB_LISTING-M/...,https://img4.idealista.com/blur/WEB_LISTING-M/...,4,4,16,6
2,107186262,145000.0,2231.0,piso,True,1,True,65.0,2,1,True,True,True,False,False,good,calle de Bellavista,""" PRECIOSO PISO REFORMADO PARA ENTRAR A VIVIR ...","Fincas Ruiz, Fincas Ruiz",876 21 04 93,['https://img4.idealista.com/blur/WEB_LISTING-...,San José,POINT (-0.87883 41.63422),757.0,https://img4.idealista.com/blur/WEB_LISTING-M/...,https://img4.idealista.com/blur/WEB_LISTING-M/...,4,4,12,5
3,106889524,125000.0,1984.0,piso,False,7,True,63.0,2,1,False,False,True,False,False,good,calle de Alonso V,"""VIVIENDA EN EL CENTRO EN LA CALLE ALONSO V, C...","Fincas Ruiz, Fincas Ruiz",876 21 04 93,['https://img4.idealista.com/blur/WEB_LISTING-...,Casco Histórico,POINT (-0.87192 41.65134),889.0,https://img4.idealista.com/blur/WEB_LISTING-M/...,https://img4.idealista.com/blur/WEB_LISTING-M/...,3,3,11,6
4,106363682,125000.0,2717.0,ático,True,3,True,46.0,2,1,True,False,True,False,False,good,calle de Terminillo,¡atención compradores! oportunidad única de ad...,"Ciz Inmobiliaria, CIZ",876 21 02 72,['https://img4.idealista.com/blur/WEB_LISTING-...,Delicias,POINT (-0.90894 41.65192),706.0,https://img4.idealista.com/blur/WEB_LISTING-M/...,https://img4.idealista.com/blur/WEB_LISTING-M/...,4,3,7,4


- Para comprobar que nuestra función de métricas financieras funciona adecuadamente, utilizaremos algunos valores de ejemplo. Esta función, `calcular_rentabilidad_inmobiliaria_wrapper`, será la base para el cálculo dinámico de las métricas en Streamlit.

In [34]:
df_rentabilidad = sr.calcular_rentabilidad_inmobiliaria_wrapper(df_scoring, porcentaje_entrada=0.2, coste_reformas=10000, comision_agencia=5000, anios=30, tin=0.03, seguro_vida=0, tipo_irpf=0.37, porcentaje_amortizacion=0.4)
df_rentabilidad.head()


=== Información de Debug ===
Shape del DataFrame de entrada: (340, 30)

 Parámeteros de entrada:
- porcentaje_entrada: 0.2
- coste_reformas: 10000
- comision_agencia: 5000
- anios: 30
- tin: 0.03
- seguro_vida: 0
- tipo_irpf: 0.37
- porcentaje_amortizacion: 0.4

Columnas del DataFrame: ['codigo', 'precio', 'precio_por_zona', 'tipo', 'exterior', 'planta', 'ascensor', 'tamanio', 'habitaciones', 'banios', 'aire_acondicionado', 'trastero', 'terraza', 'patio', 'parking', 'estado', 'direccion', 'descripcion', 'anunciante', 'contacto', 'urls_imagenes', 'distrito', 'geometry', 'alquiler_predicho', 'url_cocina', 'url_banio', 'puntuacion_cocina', 'puntuacion_banio', 'mts_cocina', 'mts_banio']

Primeras filas de columnas clave:
     precio  alquiler_predicho
0  149900.0             1180.0
1  149900.0             1186.0
2  145000.0              757.0
3  125000.0              889.0
4  125000.0              706.0

Conversión numérica para columna precio:
- Valores nulos: 0
- Valores en rango: 60000

,codigo,precio,precio_por_zona,tipo,exterior,planta,ascensor,tamanio,habitaciones,banios,aire_acondicionado,trastero,terraza,patio,parking,estado,direccion,descripcion,anunciante,contacto,urls_imagenes,distrito,geometry,alquiler_predicho,url_cocina,url_banio,puntuacion_cocina,puntuacion_banio,mts_cocina,mts_banio,Coste Total,Rentabilidad Bruta,Beneficio Antes de Impuestos,Rentabilidad Neta,Cuota Mensual Hipoteca,Cash Necesario Compra,Cash Total Compra y Reforma,Beneficio Neto,Cashflow Antes de Impuestos,Cashflow Después de Impuestos,ROCE,ROCE (Años),Cash-on-Cash Return,COCR (Años)
186,104363466,77000.0,846.0,piso,True,2,True,91.0,5,1,False,False,False,False,False,good,calle de Mariano Royo,CALLE MARIANO ROYO URIETA. - ¡SE VENDE LA NUDA...,"G.I.Z. - Gestión Inmobiliaria Zaragoza, GIZ",876 21 05 86,['https://img4.idealista.com/blur/WEB_LISTING-...,Centro,POINT (-0.8888389 41.6445102),1065.0,None,None,0,0,0,0,99700.0,12.82,8517.73,7.00,-259.71,28100.0,33100.0,6983.10,6464.39,4929.77,33.54,2.98,12.94,7.73
298,107056611,92000.0,708.0,piso,True,ND,False,130.0,4,2,False,False,False,False,False,good,Barrio Almozara,Oportunidad de inversión: inmueble sin posesió...,"Sa Roqueta Investment, Oficina SRI",871 18 55 64,['https://img4.idealista.com/blur/WEB_LISTING-...,La Almozara,POINT (-0.9041589 41.6636926),1240.0,None,None,0,0,0,0,116200.0,12.81,9950.91,7.01,-310.30,32600.0,37600.0,8148.07,7497.58,5694.74,34.93,2.86,13.37,7.48
276,106848232,92000.0,989.0,piso,True,2,False,93.0,2,2,True,False,False,False,False,good,calle de Basilio Boggiero,Findliving te presenta esta magnifica vivienda...,"Findliving Real Estate, Findliving Real Estate",876 21 09 31,['https://img4.idealista.com/blur/WEB_LISTING-...,Casco Histórico,POINT (-0.886008 41.6553579),1233.0,https://img4.idealista.com/blur/WEB_LISTING-M/...,https://img4.idealista.com/blur/WEB_LISTING-M/...,4,4,10,5,116200.0,12.73,9882.87,6.97,-310.30,32600.0,37600.0,8095.13,7429.54,5641.80,34.73,2.88,13.24,7.55
261,106798186,91500.0,1130.0,piso,ND,ND,False,81.0,4,3,False,False,False,False,False,good,calle de Italia,Inmueble sin posesión y sin acceso al interior...,"Hipoges, Hipoges",919 38 05 58,['https://img4.idealista.com/blur/WEB_LISTING-...,Delicias,POINT (-0.9011514 41.6509251),1153.0,None,None,0,0,0,0,115650.0,11.96,9114.20,6.48,-308.61,32450.0,37450.0,7495.44,6674.20,5055.44,32.59,3.07,11.91,8.40
333,105606006,60000.0,938.0,estudio,True,bj,False,64.0,0,1,False,False,False,False,False,good,"calle de Miguel Ángel Blanco, 18",Estupendo local exterior reformado como vivien...,"Particular, Rubén",ND,['https://img4.idealista.com/blur/WEB_LISTING-...,Oliver-Valdefierro,POINT (-0.9290472 41.6399239),756.0,https://img4.idealista.com/blur/WEB_LISTING-M/...,https://img4.idealista.com/blur/WEB_LISTING-M/...,3,3,9,6,81000.0,11.20,5817.77,5.96,-202.37,23000.0,28000.0,4825.93,4217.77,3225.93,27.49,3.64,9.78,10.23


- Como paso final, subiremos el dataframe completo (sin métricas de rentabilidad) a Mongo Atlas, que será desde donde, la aplicación de Streamlit extraerá los datos, calculando como parte de su código la rentabilidad, según los *inputs* del usuario.

In [41]:
sm.eliminar_coleccion(bd, 'ventafinal')
sm.subir_geodataframe_a_mongo(bd, df_scoring, 'ventafinal')

GeoDataFrame subido a la colección: ventafinal


- Con este notebook finaliza el proyecto en este repositorio. Un segundo repositorio con la aplicación de Streamlit está disponible en https://github.com/davfranco1/Streamlit-Viviendas.